<img src="./images/DLI_Header.png" style="width: 400px;">

# 3. Optimizing inference with NVIDIA FasterTransformer library 

In this lab, we are going to look at the NVIDIA FasterTransformer library and how it optimizes execution of large language models. We will use it to deploy GPT-J 6B initially using just a single GPU but afterwards taking advantage of its Tensor and Pipeline parallelism capabilities on multiple GPUs.  

We will conclude this notebook by comparing the latency between our baseline implementation using Transformers library and the FasterTransformer Tensor and Pipeline parallel deployments. In the next notebook we will look at how to serve our FasterTransformer optimized model to your customers/users using Triton Inference Server. 

To summarize, in this notebook we will: 
* Review the features of NVIDIA FasterTransformer library. 
* Learn how to build the development environment including building FasterTransformer library. 
* Learn how to prepare a checkpoint of GPT-J model (or other Transformers based model) for inference with FasterTransformer. 
* Run inference of the model on a single GPU. 
* Extend the execution to multiple GPUs using Tensor Parallelism. 
* Profile the single and multi GPU pipelines to capture information about throughput and latency. 

**[3.1 NVIDIA FasterTransformer](#3.1)<br>** 
**[3.2 Overall Inference Pipeline with NVIDIA FasterTransformer](#3.2)<br>** 
**[3.3 Download and Build NVIDIA FasterTransformer library](#3.3)<br>** 
**[3.4 Download and prepare GPT-J checkpoint](#3.4)<br>** 
**[3.5 Convert weights into FT-friendly format for the inference](#3.5)<br>** 
**[3.6 GPT-J inference using C++ bindings](#3.6)<br>** 
&nbsp;&nbsp;&nbsp;&nbsp;[3.6.1 Inference on 1 GPU ](#3.4.1)<br> 
&nbsp;&nbsp;&nbsp;&nbsp;[3.6.2 Inference on 2 GPUs ](#3.4.2)<br> 

# 3.1 NVIDIA FasterTransformer

## Introduction 

[NVIDIA’s FasterTransformer (FT)](https://github.com/NVIDIA/FasterTransformer) is a library implementing an accelerated engine for the inference of Transformer-based Neural Networks. FasterTransformer was developed with an emphasis on inference of large models also including support for multi GPU and multi node inference.  

FasterTransformer contains an optimized implementation of the Transformer block that contains both the encoder and decoder components. FasterTransformer can be used to execute both encoder-decoder architectures like T5, as well as BERT-like (encoder only) or GPT-3 like (decoder only) models. It is written in C++/CUDA and relies on cuBLAS, cuBLASlt , cuSPARSELt libraries to provide the fastest computational pipelines on GPU. 

## Large language models 

GPT and GPT-J are both variants of the transformer architecture. They do not implement the encoder module or cross multi-head attention. They are decoder only models. They both use GeLU as the activation function. In 2020, OpenAI demonstrated that using a large language model trained in self supervised way on large volume of training data can significantly improve the capacity of GPT model ([refer to the paper for more details](https://arxiv.org/abs/2005.14165)). The largest GPT-3 variant, has 175 billion parameters, which consumes about 350 GBs even when represented in half-precision. Therefore putting such a model on a single GPU is impossible making multi-gpu or even multi-node deployment a necessity. To solve the challenges of latency and memory footprint, FasterTransformer provides high efficiency kernels, optimized for memory usage, and support for model parallelism. It also comes with a wide range of other features including: 
* Number of checkpoint converters:
  * Huggingface
  * Megatron
  * Nemo Megatron
  * TensorFlow
* Support for a range of data types:
  * FP32
  * FP16
  * INT8 (weights only, PTQ for batch size 1 and 2)
* Advanced feature:
  * Multi-GPU multi-node inference
  * Dynamic random seed
  * Stop tokens
  * Beam search and sampling are both supported
  * FP32, FP16 and INT8 inference
* Frameworks
  * TensorFlow
  * PyTorch
  * C++
  * Triton backend
  
 
This section of the notebook discusses how FasterTransformer can be used for optimization of the GPT-J model. It explains the optimization workflow for both single and multi GPU deployments. 

### Tensor and Pipeline Parallelism 

Under the hood, FasterTransformer relies on MPI and NVIDIA NCCL to enable inter/intra node communication. Using this software stack, anyone can run huge Transformers in Tensor-Parallelism mode on multiple GPUs to reduce computational latency. At the same time, tensor parallelism and pipeline parallelism can be combined to execute large models with billions and trillions of parameters (which amount to terabytes of weights) in Multi-GPU and Multi-Node environments. 

We have discussed the techniques below in the lecture but let us revisit them before diving into the implementation detail: 
- Data Parallelism (DP) - is a technique used during the training process. Every GPU receives the same copy of the model but different data to process. The GPUs execute the forward pass in parallel and exchange the gradients during the backward pass allowing all the devices to make a synchronized weights update based on the average of the accumulated gradients. 
- Tensor Parallelism (TP) - is a technique used both during training and inference. Instead of splitting the data across multiple GPUs, selected layers of the model are distributed. If using Tensor Parallelism across 8 GPUs each layer affected/its tensor is split into 8 segments, each processed on a separate GPU in parallel. The results are gathered at the end of the step. 
- Pipeline Parallelism (PP) - similarly, this is a technique used both in training and inference. Here individual layers are not being split into pieces, instead they are sequentially distributed across multiple GPUs. E.g. if training a 10 layer deep neural network across 2 GPUs, the first five layers would be deployed on the first GPU and the rest on the second GPU. Each GPU is processing data sequentially and the second GPU needs to wait for results from the first GPU. 

The diagram below demonstrates the difference between Tensor and Pipeline parallelism. 

<div style="text-align:center"><img src="./images/image3.png" style="width: 1000px;"></div>


### Optimizations in FasterTransformer library 

  

FT allows us to speed up the inference pipeline achieving lower latency and higher throughput compared to the common deep learning frameworks. Below are the key optimization techniques that allow FT to achieve its performance: 
1. <b>Layer Fusion</b></br> 
During the model pre-processing stage, certain layers can be combined to form individual execution kernels. This allows for considerable reduction in GPU memory bandwidth increasing mathematical density of our model, thus accelerating computation at the inference stage. For example, all operations in the multi-head attention block can be combined into a single kernel. 
2. <b>Autoregressive models: Keys/Values caching. </b></br> 
To prevent recomputing the previous keys and values, FT allocates a buffer to store them at each step. Although the cache consumes additional memory, FT can remove the cost of recomputing those matrices but also costs associated with memory buffer allocation, and the cost of concatenation. The diagram below illustrates the process:<br/> 
<div style="text-align:center"> 
<img src="./images/KV_caching v2.PNG" style="width: 50%;position:relative;"><br/> 
<em>Keys/Values caching</em> 
</div> 
<br/><br/> 
3. <b>Memory optimization</b></br> 
Unlike traditional models like BERT, large transformer models have billions/trillions of parameters. For instance, GPT-3 175B uses 350 GB of memory even if we store in FP16 representation. FasterTransformer can reuse the memory buffer of decoder layers. Since GPt-3 is constructed of 96 layers, we only need to allocate 1/96 of the required memory. 
4. <b>Usage of MPI and NCCL to enable inter/intra node communication and support model parallelism. </b></br> 
FasterTransormer provides support for both tensor and pipeline parallelism. Tensor parallel in FasterTransformer follows the implementation of Megatron-LM. For both the self-attention block and feed forward block, FT splits the weights of the first matrix multiplication by row and splits the weights of the second matrix multiplication by column. This allows FT to execute each transformer block with just two reduction operations. For pipeline parallelism, FasterTransformer splits the whole batch of request into multiple micro batches and hides the bubble of communication. FasterTransformer will adjust the micro-batch size automatically for different cases. Users can adjust the model parallelism by modifying the gpt_config.ini file. We recommend using tensor parallel intra node and pipeline parallel inter node because tensor parallel implementation requires more NCCL communication. 
5. <b>MatMul kernel autotuning (GEMM Autotuning)</b></br> 
Matrix Multiplication is the most computationally intensive operation in transformer-based neural networks. FT uses functionality from CuBLAS and CuTLASS libraries to execute GEMM operations. It is important to know that MatMul operations can be executed in many different ways using different low-level algorithms on the “hardware” level depending on the properties of the matrices being processed. The GemmBatchedEx function implements the MatMul operation and has “cublasGemmAlgo_t”  as an input parameter. Using this parameter, we can choose different low-level algorithms executing this operation. The FasterTransformer library uses this parameter to do a real-time benchmark of all low-level algorithms and to choose the best one for the specific parameters of the model (size of the attention layers, number of attention heads, size of the hidden layer, …) and batch dimensions. Additionally, FT uses hardware-accelerated low-level functions for some parts of the network such as: __expf(), __shfl_xor_sync() 
<div style="text-align:center"> 
<img src="./images/kernel autotuning.png" style="width: 20%;position:relative;"><br/> 
<em>Kernel autotuning</em> 
</div> 
<br/><br/> 
6. <b>Reduced precision inference</b></br> 
FT has kernels that support inference using low-precision input data in fp16 and int8. Both these regimes allow acceleration due to the reduction in data transfer and required memory. At the same time, int8 and fp16 computations can be hardware accelerated using TensorCores (available on all GPU architectures starting from Volta).  
7. <b>Other optimizations include:</b></br> 
    - Efficient C++ implementation of BeamSearch  
    - Optimized all-reduce implementation of eight-way Tensor Parallelism. 

# 3.2 Overall Inference Pipeline with NVIDIA FasterTransformer 
The diagram listed below lists all the steps involved in using FasterTransformer library to deploy large models to production. In the next section, we will go through them one at a time. 

<div style="text-align:center">
<img src="./images/FT_pipeline.PNG" style="width: 70%"/>
</div>

## 3.3 Download and Build NVIDIA FasterTransformer library

Starting with Triton 23.10 release, Triton includes a container with the TensorRT-LLM Backend and the Python Backend. This container should have everything needed to run a TensorRT-LLM model. You can find this container [NGC](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/tritonserver).

### Step 1: Cloning FasterTransformer library from GitHub
Execute the below command to clone the code repository. To ensure consistent student experience, we will use a specific branch.

In [ ]:
!git clone https://github.com/NVIDIA/FasterTransformer.git
%cd FasterTransformer
!git checkout 6b3fd4392831f972d48127e881a048567dd92811

### Step 2: Building the FT library

#### Install dependencies

We should install additional libraries like `cmake` for build automation, and we also should install compression libraries "zstd" and "libz-dev" that will allow decompressing weights of the GPT-J 6b model. </br>


In [ ]:
!apt update
!DEBIAN_FRONTEND=noninteractive apt install -y cmake xz-utils zstd libz-dev

#### Build library

We downloaded the FasterTransformer code from Github. We will build this library manually as well as additional scripts that allow us to convert pre-trained files of the GPT-J into FT binary format. Let us create a `build` directory that will be used as the main directory for the built process. 

In [ ]:
!mkdir -p build
%cd build 

Let us build the library with the arg `-DBUILD_MULTI_GPU=ON ` that includes in the build pipeline multi-GPU cases like `GPT-J`. Please note, this step can take several minutes. Feel free to take a short break as you wait for the build process to complete.

In [ ]:
!git submodule init && git submodule update
!cmake -DSM=xx -DCMAKE_BUILD_TYPE=Release -DBUILD_MULTI_GPU=ON ..
!make -j32

Let's have a look at all pre-built binary files with examples and useful utilities.

In [ ]:
print("List of FasterTransformer examples:")
!ls ./bin

Looks like the FasterTransformer library was successfully built and ready for the inference of our models.

## 3.4 Step 3: Download and prepare GPT-J checkpoint

Let's see how to execute inference of a GPT model using C programming language. Before we start, we need to convert the checkpoints from the TensorFlow or PyTorch format to a binary representation that can be consumed by FasterTransformer C API. Currently there are no published checkpoints released by OpenAI. Fortunately, FasterTransformer provides several converters which will allow to consume models in several formats. FasterTransformer can consume Megatron-LM (which is trained by PyTorch) checkpoints. Another option includes using the OpenAI GPT-2 model format or consume Transformers library based checkpoints. In this example we will continue to use GPT-J 6B checkpoint.

We already downloaded the weights for you, but you can always uncomment the lines below to download if desired:

In [ ]:
# Go to the main course dir
%cd /dli/task

In [ ]:
# We already downloaded the weights for you.
# Please uncomment the below to download from scratch on your own system.
# !wget https://the-eye.eu/public/AI/GPT-J-6B/step_383500.tar.zstd
!tar -axf ./weights/gpt-j/ft/step_383500_slim.tar.zstd -C ./model_repository/

Let's have a look at downloaded weights.

In [ ]:
!ls ./model_repository/step_383500/

Weights are split into multiple shards. That is how they were pretrained with the Megatron framework. Shards are parts of the weights that were placed onto different devices in the process of training.

On top of the GPT-J model we will use Transformers’ library implementation of tokenizers as well as encoding/decoding components. We will also need the Vocab and Merge table files that will be needed at the inference stage to initialize the tokenizer.

In [ ]:
!wget https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json -P models
!wget https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt -P models

## 3.5 Step 4: Convert weights into FT format

FasterTransformer provides a number of tools/scripts for format conversion. </br> 
For GPT-J weights, we can use the following script `FasterTransformer/examples/pytorch/gptj/utils/gptj_ckpt_convert.py`.</br> 
The converter requires the following arguments: 

1. `--ckpt-dir`: The path of megatron model
2. `--output-dir`: The output path of converted model
3. `--n-inference-gpus`: The tensor parallel size of the pipeline

Please note that because of the file sizes involved, the conversion can take a couple minutes.

In [ ]:
# get to directoy with build FasterTransformer library 
%cd FasterTransformer/build

In [ ]:
# Converting weights for 1 GPU without parallelism
!python3 ../examples/pytorch/gptj/utils/gptj_ckpt_convert.py \
                                --output-dir ../../models/j6b_ckpt \
                                --ckpt-dir ../../model_repository/step_383500/ \
                                --n-inference-gpus 1

Let us have a look at the generated files. The conversion script has re-combined sharded weights into groups of weights that will be placed onto GPUs that we want to use for inference. For now, we have configured the pipeline for a single GPU execution.

In [ ]:
!ls ../../models/j6b_ckpt/

## 3.6 Step 5: Kernel autotuning. Seeking the fastest CUDA kernels for our model

Let us run `./FasterTransformer/build/bin/gpt_gemm` binary file that we have built earlier. This file takes as an input the parameters of our model as well as additional configuration parameters, namely:  
* `batch_size`  
* `beam_width`  
* `max_input_len`  
* `head_number`  
* `size_per_head` 
* `inter_size` 
* `vocab_size`  
* `data_type`  
* `tensor_para_size` 

All these parameters are needed to simulate the execution of the neural network and to identify the fastest algorithms to use during the inference process. We will need to set up these parameters manually. For the GPT-J model, the FasterTransformer team has prepared a `../examples/cpp/gptj/gptj_config.ini` config that contains the information about the GPT-J model as well as parameters required for inference. </br> 

<b>All these tests need to be run on the GPU that will be used for the inference later as those optimizations are GPU specific.</b> 

In [ ]:
!CUDA_VISIBLE_DEVICES=1  ./bin/gpt_gemm 1 1 128 16 256 16384 50256 1 1

As a result of kernel autotuning, we received a `gemm_config.in` file in our directory that contains information about algorithms which were tested and the execution time for each algorithm.  This configuration file will be used later by the FT library in any of our inference examples whether through bindings or C++ code.

# 3.6 Step 6: GPT-J inference using C++ bindings 

## 3.6.1 Inference on 1 GPU 

Before we start, we have to look one more time at the configuration produced earlier: `./FasterTransformer/examples/cpp/gptj/gptj_config.ini`. 

It contains information about our GPT-J model like the number of GPT decoder layers, number of heads, number of hidden neurons in MLP layers, etc. At the same time, it contains hyperparameters for our inference like the batch size, search temperature, top_k, top_p, beam_search parameters, as well as the precision mode (e.g. FP16 or FP32).

**TODO**: Please modify the [gptj_config.ini](FasterTransformer/examples/cpp/gptj/gptj_config.ini) to make inference execution consistent to the previous notebook:</br>
* max_batch_size=<b>1</b> 
* top_k=<b>50</b> 
* top_p=<b>1</b> 
* is_half=<b>1</b> #set to 1 to run model in FP16 mode.
* tensor_para_size=<b>1</b> # we will run on a single GPU without Tensor Parallelism
* model_dir=<b>../../models/j6b_ckpt/</b> # Path to model weights
* request_batch_size=<b>1</b>
* request_output_len=<b>120</b> # Output length

All other parameters should be left unmodified for a fair comparison.

Let's run the pre-built script `./FasterTransformer/build/bin/gptj_example`. It was built by us from C++ file `./FasterTransformer/examples/cpp/gptj/gptj_example.cpp`. </br>
This file takes weights that we prepared and the config from `./FasterTransformer/examples/cpp/gptj/gptj_config.ini`. This binary file takes as input tokens from `./FasterTransformer/examples/cpp/gptj/start_ids.csv` as a prompt and generates an `out` text file that contains tokens generated in the process of work of GPT-J.

`CUDA_VISIBLE_DEVICES` environment variable allows you to choose what GPU will be used for current task/process/script. We have 4 GPUs on our system (0-3 numerations starts with 0) and we will use this variable to run current example onto GPU 1 (2nd GPU). We will use GPU 1 because previous task (Notebook 02) was deployed to GPU 0 and model is already there if you have not stopped the Jupyter kernel.

In [ ]:
!CUDA_VISIBLE_DEVICES=1 ./bin/gptj_example

We see that the inference time of the sentence with the same length is <b>1.8 seconds</b>, which is <b>x3.5</b> times faster than the default PyTorch + Transformers pipeline! Please note that since we are using different GPUs between classes, your performance can be different than the above.</br>
Let's decode the `out` file and see what was generated by our neural network.

In [ ]:
!python3 ../examples/pytorch/gpt/utils/gpt_token_converter.py \
                       --vocab_file=../../models/gpt2-vocab.json  \
                       --bpe_file=../../models/gpt2-merges.txt

Users can see the details of arguments in `examples/cpp/multi_gpu_gpt/gpt_config.ini`. It controls the model path, model size, tensor parallelism size, and some hyper-parameters.

## 3.6.2 Inference on 2 GPUs

We can run the same inference step but almost two times faster using Tensor-Parallelism.

Let us run the inference of the GPT-J in Tensor-Parallel=2 mode. This means that we must split our weights into 2 parts and place them onto multiple GPUs connected with NVLink. FasterTransformer hides the complexity of model distribution, and we can enable this functionality by setting the appropriate export arguments. 

For the weights preparation, we will use the same script, but set up `--n-inference-gpus` to `2` to prepare weights for TP=2 mode. Because of the size of the files involved, this step will take a couple of minutes. 

In [ ]:
# Converting weights for 2 GPU without parallelism
!python3 ../examples/pytorch/gptj/utils/gptj_ckpt_convert.py \
                                --output-dir ../../models/j6b_ckpt \
                                --ckpt-dir ../../model_repository/step_383500/ \
                                --n-inference-gpus 2

Let us have a look at the generated files. We have re-combined sharded weights into two groups of weights that will be placed onto two GPUs in our tensor parallel deployment. 

In [ ]:
!ls ../../models/j6b_ckpt/

Now we have to run kernel-autotuning again to find the fastest kernels for the 2xGPU mode. First, let's run inference with the kernels that we already found for 1xGPU mode on the previous step. We do this non-optimal inference just to compare results later with kernel-autotuning to see the acceleration this operation gives us. 

In [ ]:
!CUDA_VISIBLE_DEVICES=1,2 mpirun -n 2 --allow-run-as-root ./bin/gptj_example

The inference with the non-optimal kernels is <b>~1.4s</b>. We achieved acceleration in comparison with 1xGPU mode, but it can be better. Now we have to do kernel-autotuning for Tp 2 regime because there is some probability that other algorithms may work better. We use the same parameters for our NN, but at this time we should set `tensor_para_size = 2` 

 We will use GPU 1 because previous task (Notebook 02) was started on GPU 0 and the model is already there if you haven't stopped the Jupyter kernel. So we just avoiding OOM error. 

In [ ]:
!CUDA_VISIBLE_DEVICES=1  ./bin/gpt_gemm 1 1 128 16 256 16384 50256 1 2

We remember from the first lab that we have 4 GPUs available on our server, so let's use 2 of them to start inference in TP=2 regime

Let's configure `./FasterTransformer/examples/cpp/gptj/gptj_config.ini` to run GPT-J C++ inference case.</br>
We need to open this file in JupyterLab and update these lines to make our inference case similar to the inference pipeline we had the previous time but with 2 GPUs:</br>
* tensor_para_size=<b>2</b> #we run on 2xGPUs with Tensor-Parallelism mode

All other parameters should be the same as in the previous run with FT.
 We will use GPUs 2 and 3 (numbers 1 and 2) because previous task (Notebook 02) was started on GPU 0 and the model is already there if you haven't stopped the Jupyter kernel. So we just avoiding OOM error. Let's run inference with optimized kernels now.

In [ ]:
!CUDA_VISIBLE_DEVICES=1,2 mpirun -n 2 --allow-run-as-root ./bin/gptj_example

We see that the inference time of the sentence with the same length is <b>1.25 seconds </b> which is <b>x5</b> times faster than the default PyTorch + Transformers pipeline! Of course, we used 2 GPUs here, but this demonstrates how we can achieve lower latency by adding more GPUs to inference.</br>
Let's decode the `out` file and see what was generated by our neural network.

In [ ]:
!python3 ../examples/pytorch/gpt/utils/gpt_token_converter.py \
                               --vocab_file=../../models/gpt2-vocab.json  \
                               --bpe_file=../../models/gpt2-merges.txt

## Results

Let's compare the results. We get 3.5x acceleration using FasterTransformer in TP=1 mode in comparison with Transformers library based implementation and <b>5x</b> acceleration with two way tensor parallelism. We achieved a significant performance boost. You can use this C++ example as well as PyTorch/TF bindings in your inference pipelines.</br> 
At the same time, if you wish to have more unified inference engine/serving solution we will look at it in the next notebook.

<h2 style="color:green;">Congratulations!</h2>

Please proceed on to [Inference of the GPT-J 6b model with Triton Inference server and FasterTransformer as a backend.](04_FTAndTritonRunRemoteInferenceOfTheGPT-J.ipynb)
